In [30]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error

In [31]:
import os
# Office
os.chdir('C:\\Users\\prudi\\Documents\\GitHub\\Code\\NITDATA\\Algorithms\\codesnippetsforbaggingandboostingalgorithms')
data = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

class PreProcessing(BaseEstimator, TransformerMixin):


    def __init__(self):
        pass

    
    ## Here this model will be called with a variable (df), which is a dataframe
    def datapreprocessing(self, df):
        
        df = df.reset_index(drop=True)
        
        df.loc[df['Gender'].isnull(),'Gender']='Trasgender'
        df.loc[df['Dependents'].isnull(),'Dependents']='0'
        df.loc[df['Education'].isnull(),'Education']='No Education'
        df.loc[df['Married'].isnull(),'Married']='No'
        df.loc[df['Self_Employed'].isnull(),'Self_Employed']='No'
        df.loc[df['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=0
        df.loc[df['Credit_History'].isnull(),'Credit_History']=0
        df.loc[df['LoanAmount'].isnull(),'LoanAmount']=146.867     
        df['Credit_History']=df['Credit_History'].astype(str)
        
        
        ## Creating a Instance for the minmax Pickle File
        MinMaxpickle_in = open("minmax_pickle.pkl","rb")
        MinmaxScaler_dict = pickle.load(MinMaxpickle_in)
        
        ## Applying the pickle file
        df['ApplicantIncome']=pd.DataFrame(MinmaxScaler_dict['ApplicantIncome'].transform(df[['ApplicantIncome']]))
        
        ## Creating a Instance for the minmax Pickle File
        standardscaler_in = open("standardscaler_pickle.pkl","rb")
        standardscaler_dict = pickle.load(standardscaler_in)
        
        ## Applying the pickle file
        df['LoanAmount']=pd.DataFrame(standardscaler_dict['LoanAmount'].transform(df[['LoanAmount']]))
        
        ## Creating a Instance for the LabelEncoder Pickle File
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Education']=pd.DataFrame(Labelencoder_dict['Education'].transform(df[['Education']]))
        df['Property_Area']=pd.DataFrame(Labelencoder_dict['Property_Area'].transform(df[['Property_Area']]))
        df['Credit_History']=pd.DataFrame(Labelencoder_dict['Credit_History'].transform(df[['Credit_History']]))
        df['Dependents']=pd.DataFrame(Labelencoder_dict['Dependents'].transform(df[['Dependents']]))
        
        ## Creating a Instance for the binarizer Pickle File
        Onehotpickle_in = open("binarizer_pickle.pkl","rb")
        Onehot_dict = pickle.load(Onehotpickle_in)
        
        ## Applying the pickle file
        d1=pd.DataFrame(Onehot_dict['Self_Employed'].transform(df[['Self_Employed']]))
        d1.columns=['Self_Employed_0']
        d2=pd.DataFrame(Onehot_dict['Married'].transform(df[['Married']]))
        d2.columns=['Married_0']
        d3=pd.DataFrame(Onehot_dict['Gender'].transform(df[['Gender']]))
        d3.columns=['Gender_0','Gender_1','Gender_2']
    
        df=df.drop(['Self_Employed','Married','Gender','Loan_ID'],axis=1)
        df=pd.concat([df,d1,d2,d3],axis=1)
        
        return df

    def definingvalues(self, df, y=None,**fit_params):
        self.term_mean_ = df['Loan_Amount_Term'].mean()
        self.amt_mean_ = df['LoanAmount'].mean()
        return self
    
    def encodingTargetVariable(self,df):
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Loan_Status']=pd.DataFrame(Labelencoder_dict['Loan_Status'].transform(df[['Loan_Status']]))
        
        return df.as_matrix()

In [33]:
data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [34]:
pred_var = ['Loan_ID','Gender','Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area']
X_train, X_test, y_train, y_test = train_test_split(data[pred_var], data['Loan_Status'], test_size=0.30, random_state=42)

In [35]:
## Creating an instance for the class
preprocess = PreProcessing()

In [36]:
# Calling the fit function present in the class Preprocessing
preprocess.definingvalues(X_train)

PreProcessing()

In [37]:
## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_train dataset
data_transformed_xtrain = preprocess.datapreprocessing(X_train)

pred_var = ['Dependents', 'Education', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area', 'Self_Employed_0', 'Married_0', 'Gender_0', 'Gender_1','Gender_2']


data_transformed_xtrain=pd.DataFrame(data_transformed_xtrain,columns=pred_var)
data_transformed_xtrain.head()

print(data_transformed_xtrain.shape)
print(X_train.shape)

(429, 13)
(429, 12)


In [38]:
data_transformed_xtrain.columns

Index(['Dependents', 'Education', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Self_Employed_0', 'Married_0', 'Gender_0', 'Gender_1', 'Gender_2'],
      dtype='object')

In [39]:
## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_test dataset
data_transformed_xtest = preprocess.datapreprocessing(X_test)

# Converting the matrix to a dataframe
data_transformed_xtest=pd.DataFrame(data_transformed_xtest,columns=pred_var)
data_transformed_xtest.head()

,Dependents,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Self_Employed_0,Married_0,Gender_0,Gender_1,Gender_2
0,0,0,0.681284,0.0,1.426199,360.0,1,1,0,1,0,1,0
1,0,0,0.317267,0.0,-0.108986,360.0,0,1,0,1,0,1,0
2,2,0,0.306361,1447.0,0.329638,360.0,1,0,0,1,0,1,0
3,0,0,0.209732,0.0,-1.033230,360.0,1,0,0,0,1,0,0
4,0,0,0.195851,0.0,-1.048895,360.0,1,1,0,0,0,1,0


In [40]:
y_train=pd.DataFrame(y_train).reset_index(drop=True)
y_train.columns=['Loan_Status']

y_test=pd.DataFrame(y_test).reset_index(drop=True)
y_test.columns=['Loan_Status']

y_train=preprocess.encodingTargetVariable(y_train)
y_test=preprocess.encodingTargetVariable(y_test)

Befor you run this code 

You have to install below libraries

pip install catboost

In [41]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()

In [42]:
categorical_features_indices=np.where(data_transformed_xtrain.dtypes != np.float)[0]

In [43]:
model.fit(data_transformed_xtrain,y_train,eval_set=(data_transformed_xtest, y_test))

Learning rate set to 0.025711
0:	learn: 0.6824786	test: 0.6847245	best: 0.6847245 (0)	total: 10.7ms	remaining: 10.7s
1:	learn: 0.6734887	test: 0.6769057	best: 0.6769057 (1)	total: 12.7ms	remaining: 6.34s
2:	learn: 0.6640059	test: 0.6680351	best: 0.6680351 (2)	total: 15.5ms	remaining: 5.16s
3:	learn: 0.6548542	test: 0.6608290	best: 0.6608290 (3)	total: 18.4ms	remaining: 4.57s
4:	learn: 0.6461697	test: 0.6542102	best: 0.6542102 (4)	total: 21.2ms	remaining: 4.22s
5:	learn: 0.6358901	test: 0.6480193	best: 0.6480193 (5)	total: 23.7ms	remaining: 3.93s
6:	learn: 0.6277652	test: 0.6422095	best: 0.6422095 (6)	total: 26ms	remaining: 3.69s
7:	learn: 0.6200362	test: 0.6358066	best: 0.6358066 (7)	total: 27.9ms	remaining: 3.46s
8:	learn: 0.6133247	test: 0.6304716	best: 0.6304716 (8)	total: 29.7ms	remaining: 3.27s
9:	learn: 0.6061231	test: 0.6258797	best: 0.6258797 (9)	total: 31.4ms	remaining: 3.11s
10:	learn: 0.5997852	test: 0.6211241	best: 0.6211241 (10)	total: 33ms	remaining: 2.97s
11:	learn: 0.59

97:	learn: 0.3909696	test: 0.5551217	best: 0.5481747 (67)	total: 153ms	remaining: 1.41s
98:	learn: 0.3898695	test: 0.5553806	best: 0.5481747 (67)	total: 155ms	remaining: 1.41s
99:	learn: 0.3885882	test: 0.5549143	best: 0.5481747 (67)	total: 156ms	remaining: 1.41s
100:	learn: 0.3878250	test: 0.5555831	best: 0.5481747 (67)	total: 158ms	remaining: 1.41s
101:	learn: 0.3869452	test: 0.5562126	best: 0.5481747 (67)	total: 159ms	remaining: 1.4s
102:	learn: 0.3859286	test: 0.5550773	best: 0.5481747 (67)	total: 161ms	remaining: 1.4s
103:	learn: 0.3847684	test: 0.5552284	best: 0.5481747 (67)	total: 162ms	remaining: 1.4s
104:	learn: 0.3838606	test: 0.5552008	best: 0.5481747 (67)	total: 164ms	remaining: 1.4s
105:	learn: 0.3822898	test: 0.5557903	best: 0.5481747 (67)	total: 165ms	remaining: 1.39s
106:	learn: 0.3818052	test: 0.5558147	best: 0.5481747 (67)	total: 167ms	remaining: 1.39s
107:	learn: 0.3808872	test: 0.5560093	best: 0.5481747 (67)	total: 168ms	remaining: 1.39s
108:	learn: 0.3800884	test: 

208:	learn: 0.2971673	test: 0.5704913	best: 0.5481747 (67)	total: 306ms	remaining: 1.16s
209:	learn: 0.2966115	test: 0.5709499	best: 0.5481747 (67)	total: 308ms	remaining: 1.16s
210:	learn: 0.2961270	test: 0.5711812	best: 0.5481747 (67)	total: 309ms	remaining: 1.16s
211:	learn: 0.2952701	test: 0.5716901	best: 0.5481747 (67)	total: 311ms	remaining: 1.15s
212:	learn: 0.2944828	test: 0.5725483	best: 0.5481747 (67)	total: 312ms	remaining: 1.15s
213:	learn: 0.2941615	test: 0.5726929	best: 0.5481747 (67)	total: 313ms	remaining: 1.15s
214:	learn: 0.2932773	test: 0.5721132	best: 0.5481747 (67)	total: 315ms	remaining: 1.15s
215:	learn: 0.2930491	test: 0.5722038	best: 0.5481747 (67)	total: 316ms	remaining: 1.15s
216:	learn: 0.2925717	test: 0.5722156	best: 0.5481747 (67)	total: 317ms	remaining: 1.15s
217:	learn: 0.2917784	test: 0.5723709	best: 0.5481747 (67)	total: 319ms	remaining: 1.14s
218:	learn: 0.2912882	test: 0.5726642	best: 0.5481747 (67)	total: 320ms	remaining: 1.14s
219:	learn: 0.2905435

315:	learn: 0.2293031	test: 0.5852593	best: 0.5481747 (67)	total: 454ms	remaining: 983ms
316:	learn: 0.2289134	test: 0.5849409	best: 0.5481747 (67)	total: 456ms	remaining: 982ms
317:	learn: 0.2283765	test: 0.5853885	best: 0.5481747 (67)	total: 457ms	remaining: 980ms
318:	learn: 0.2275766	test: 0.5854075	best: 0.5481747 (67)	total: 459ms	remaining: 979ms
319:	learn: 0.2271181	test: 0.5854895	best: 0.5481747 (67)	total: 460ms	remaining: 977ms
320:	learn: 0.2269880	test: 0.5858066	best: 0.5481747 (67)	total: 461ms	remaining: 976ms
321:	learn: 0.2266723	test: 0.5856696	best: 0.5481747 (67)	total: 463ms	remaining: 974ms
322:	learn: 0.2264556	test: 0.5857115	best: 0.5481747 (67)	total: 464ms	remaining: 973ms
323:	learn: 0.2256609	test: 0.5858020	best: 0.5481747 (67)	total: 465ms	remaining: 971ms
324:	learn: 0.2250361	test: 0.5863004	best: 0.5481747 (67)	total: 467ms	remaining: 970ms
325:	learn: 0.2249394	test: 0.5864262	best: 0.5481747 (67)	total: 469ms	remaining: 969ms
326:	learn: 0.2242761

428:	learn: 0.1822502	test: 0.6002510	best: 0.5481747 (67)	total: 608ms	remaining: 809ms
429:	learn: 0.1819833	test: 0.6003110	best: 0.5481747 (67)	total: 609ms	remaining: 808ms
430:	learn: 0.1814589	test: 0.6005117	best: 0.5481747 (67)	total: 611ms	remaining: 806ms
431:	learn: 0.1809699	test: 0.6003209	best: 0.5481747 (67)	total: 612ms	remaining: 805ms
432:	learn: 0.1803114	test: 0.6002076	best: 0.5481747 (67)	total: 614ms	remaining: 803ms
433:	learn: 0.1801049	test: 0.6001557	best: 0.5481747 (67)	total: 615ms	remaining: 802ms
434:	learn: 0.1797525	test: 0.6001203	best: 0.5481747 (67)	total: 616ms	remaining: 800ms
435:	learn: 0.1791006	test: 0.6003402	best: 0.5481747 (67)	total: 618ms	remaining: 799ms
436:	learn: 0.1786970	test: 0.6008239	best: 0.5481747 (67)	total: 619ms	remaining: 797ms
437:	learn: 0.1780497	test: 0.6013454	best: 0.5481747 (67)	total: 620ms	remaining: 796ms
438:	learn: 0.1774340	test: 0.6014412	best: 0.5481747 (67)	total: 622ms	remaining: 794ms
439:	learn: 0.1772998

539:	learn: 0.1440269	test: 0.6232769	best: 0.5481747 (67)	total: 755ms	remaining: 643ms
540:	learn: 0.1438146	test: 0.6235867	best: 0.5481747 (67)	total: 757ms	remaining: 642ms
541:	learn: 0.1436330	test: 0.6237333	best: 0.5481747 (67)	total: 758ms	remaining: 640ms
542:	learn: 0.1435436	test: 0.6239653	best: 0.5481747 (67)	total: 759ms	remaining: 639ms
543:	learn: 0.1431331	test: 0.6243054	best: 0.5481747 (67)	total: 761ms	remaining: 638ms
544:	learn: 0.1428515	test: 0.6245806	best: 0.5481747 (67)	total: 762ms	remaining: 636ms
545:	learn: 0.1425769	test: 0.6247357	best: 0.5481747 (67)	total: 763ms	remaining: 635ms
546:	learn: 0.1423558	test: 0.6247801	best: 0.5481747 (67)	total: 765ms	remaining: 633ms
547:	learn: 0.1419354	test: 0.6254662	best: 0.5481747 (67)	total: 766ms	remaining: 632ms
548:	learn: 0.1417649	test: 0.6254967	best: 0.5481747 (67)	total: 767ms	remaining: 630ms
549:	learn: 0.1415733	test: 0.6251342	best: 0.5481747 (67)	total: 769ms	remaining: 629ms
550:	learn: 0.1413316

658:	learn: 0.1129312	test: 0.6460617	best: 0.5481747 (67)	total: 913ms	remaining: 473ms
659:	learn: 0.1129109	test: 0.6460352	best: 0.5481747 (67)	total: 915ms	remaining: 471ms
660:	learn: 0.1128632	test: 0.6462130	best: 0.5481747 (67)	total: 916ms	remaining: 470ms
661:	learn: 0.1126613	test: 0.6461351	best: 0.5481747 (67)	total: 918ms	remaining: 469ms
662:	learn: 0.1126024	test: 0.6461989	best: 0.5481747 (67)	total: 919ms	remaining: 467ms
663:	learn: 0.1124325	test: 0.6463883	best: 0.5481747 (67)	total: 921ms	remaining: 466ms
664:	learn: 0.1123322	test: 0.6462190	best: 0.5481747 (67)	total: 922ms	remaining: 464ms
665:	learn: 0.1121254	test: 0.6464472	best: 0.5481747 (67)	total: 923ms	remaining: 463ms
666:	learn: 0.1119100	test: 0.6468065	best: 0.5481747 (67)	total: 925ms	remaining: 462ms
667:	learn: 0.1115408	test: 0.6477958	best: 0.5481747 (67)	total: 926ms	remaining: 460ms
668:	learn: 0.1112912	test: 0.6474438	best: 0.5481747 (67)	total: 928ms	remaining: 459ms
669:	learn: 0.1110389

769:	learn: 0.0954286	test: 0.6638847	best: 0.5481747 (67)	total: 1.06s	remaining: 318ms
770:	learn: 0.0952651	test: 0.6640551	best: 0.5481747 (67)	total: 1.06s	remaining: 317ms
771:	learn: 0.0951499	test: 0.6643874	best: 0.5481747 (67)	total: 1.07s	remaining: 315ms
772:	learn: 0.0948621	test: 0.6651640	best: 0.5481747 (67)	total: 1.07s	remaining: 314ms
773:	learn: 0.0947408	test: 0.6650627	best: 0.5481747 (67)	total: 1.07s	remaining: 312ms
774:	learn: 0.0946944	test: 0.6651156	best: 0.5481747 (67)	total: 1.07s	remaining: 311ms
775:	learn: 0.0946017	test: 0.6650362	best: 0.5481747 (67)	total: 1.07s	remaining: 310ms
776:	learn: 0.0943418	test: 0.6648877	best: 0.5481747 (67)	total: 1.07s	remaining: 308ms
777:	learn: 0.0941650	test: 0.6652025	best: 0.5481747 (67)	total: 1.07s	remaining: 307ms
778:	learn: 0.0940033	test: 0.6653858	best: 0.5481747 (67)	total: 1.08s	remaining: 305ms
779:	learn: 0.0939140	test: 0.6653115	best: 0.5481747 (67)	total: 1.08s	remaining: 304ms
780:	learn: 0.0937683

880:	learn: 0.0807929	test: 0.6827333	best: 0.5481747 (67)	total: 1.21s	remaining: 164ms
881:	learn: 0.0805409	test: 0.6831090	best: 0.5481747 (67)	total: 1.21s	remaining: 163ms
882:	learn: 0.0804541	test: 0.6833914	best: 0.5481747 (67)	total: 1.22s	remaining: 161ms
883:	learn: 0.0802445	test: 0.6844039	best: 0.5481747 (67)	total: 1.22s	remaining: 160ms
884:	learn: 0.0801329	test: 0.6846764	best: 0.5481747 (67)	total: 1.22s	remaining: 158ms
885:	learn: 0.0799655	test: 0.6850660	best: 0.5481747 (67)	total: 1.22s	remaining: 157ms
886:	learn: 0.0798694	test: 0.6854052	best: 0.5481747 (67)	total: 1.22s	remaining: 156ms
887:	learn: 0.0797578	test: 0.6853990	best: 0.5481747 (67)	total: 1.22s	remaining: 154ms
888:	learn: 0.0797319	test: 0.6854401	best: 0.5481747 (67)	total: 1.22s	remaining: 153ms
889:	learn: 0.0795089	test: 0.6849855	best: 0.5481747 (67)	total: 1.23s	remaining: 152ms
890:	learn: 0.0793043	test: 0.6855156	best: 0.5481747 (67)	total: 1.23s	remaining: 150ms
891:	learn: 0.0792212

991:	learn: 0.0688754	test: 0.6996157	best: 0.5481747 (67)	total: 1.36s	remaining: 11ms
992:	learn: 0.0687973	test: 0.6997221	best: 0.5481747 (67)	total: 1.36s	remaining: 9.62ms
993:	learn: 0.0687899	test: 0.6996996	best: 0.5481747 (67)	total: 1.37s	remaining: 8.25ms
994:	learn: 0.0686778	test: 0.7000385	best: 0.5481747 (67)	total: 1.37s	remaining: 6.87ms
995:	learn: 0.0685588	test: 0.7002163	best: 0.5481747 (67)	total: 1.37s	remaining: 5.5ms
996:	learn: 0.0684111	test: 0.7003399	best: 0.5481747 (67)	total: 1.37s	remaining: 4.12ms
997:	learn: 0.0683993	test: 0.7003382	best: 0.5481747 (67)	total: 1.37s	remaining: 2.75ms
998:	learn: 0.0682456	test: 0.7009591	best: 0.5481747 (67)	total: 1.37s	remaining: 1.37ms
999:	learn: 0.0681952	test: 0.7011427	best: 0.5481747 (67)	total: 1.37s	remaining: 0us

bestTest = 0.5481747079
bestIteration = 67

Shrink model to first 68 iterations.


In [44]:
model.score(data_transformed_xtrain,y_test)

0.0

In [45]:
model.predict(data_transformed_xtrain)

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,

### This is how we should execute

In [46]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()
categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(x_train,y_train,cat_features=([ 0,  1, 2, 3, 4, 10]),eval_set=(x_test, y_test))
model.score(x_test,y_test)
0.80540540540540539

NameError: name 'df' is not defined